<center>
<p><img src="https://www.gob.mx/cms/uploads/image/file/179499/outstanding_quienes-somos.jpg" width="300">
</p>



# Curso *Machine Learning con uso de pandas, scikit learn y libretas jupyter*

# Curvas de aprendizaje y búsqueda ávida de parámetros


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="80">
</p>
</center>



## Generando datos de aprendizaje

In [1]:
import numpy as np
import matplotlib.pyplot as pl

rng = np.random.RandomState(0)

X = 5 * rng.rand(10000, 1)
y = np.sin(X).ravel()

# Se agrega un ruido a los datos
y[::5] += 3 * (0.5 - rng.rand(X.shape[0] // 5))

X_plot = np.linspace(0, 5, 100000)[:, None]

## Usando `grid search` para encontrar el mejor modelo

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

train_size = 100

enr = GridSearchCV(
  ElasticNet(),
  param_grid={
    "alpha": np.logspace(-2, 2, 5),
    "l1_ratio": np.logspace(-2, 2, 5)
  }
)

svr = GridSearchCV(
  SVR(kernel='rbf', gamma=0.1),
  param_grid={
    "C": [1e0, 1e1, 1e2, 1e3],
    "gamma": np.logspace(-2, 2, 5)
  }
)

kr = GridSearchCV(
  KernelRidge(kernel='rbf', gamma=0.1),
  param_grid={
    "alpha": [1e0, 0.1, 1e-2, 1e-3],
    "gamma": np.logspace(-2, 2, 5)
  }
)

enr.fit(X[:train_size], y[:train_size])
svr.fit(X[:train_size], y[:train_size])
kr.fit(X[:train_size], y[:train_size])

print(f"Para Elastic Net, los menjores parámetros son {enr.best_params_}")
print(f"Para SVR, los menjores parámetros son {svr.best_params_}")
print(f"Para Kernel Ridge, los menjores parámetros son {kr.best_params_}")

sv_ratio = svr.best_estimator_.support_.shape[0] / train_size
print("Support vector ratio: %.3f" % sv_ratio)

In [ ]:
y_enr = enr.predict(X_plot)
y_svr = svr.predict(X_plot)
y_kr = kr.predict(X_plot)

sv_ind = svr.best_estimator_.support_

pl.figure(figsize=(15,7))
pl.scatter(
  X[sv_ind], y[sv_ind], 
  c='r', s=50, label='SVR support vectors', zorder=2, edgecolors=(0, 0, 0)
)
pl.scatter(
  X[:100], y[:train_size], 
  c='k', label='data', zorder=1, edgecolors=(0, 0, 0)
)
pl.plot(
  X_plot, y_enr, 
  c='r', label='Elastic Net Regression'
)
pl.plot(
  X_plot, y_svr, 
  c='r', label='Support Vector Regression'
)
pl.plot(
  X_plot, y_kr, 
  c='g', label='Kernel Ridge Regression'
)
pl.xlabel('data')
pl.ylabel('target')
pl.title('Elastic Net SVR vs Kernel Ridge')
pl.legend()

## Una función para la curva de aprendizaje

En esta función calculamos y graficamos 3 indicadores:

1. La curva de aprendizaje tal cual
2. El tiempo de entrenamiento conforme crece el conjunto de datos
3. La relación entre tiempo de entrenamiento 

In [44]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(
    estimator, X, y, train_sizes=np.linspace(.1, 1.0, 5), 
    score='neg_mean_absolute_error', cv=None, n_jobs=None
):
    """
    Genera una gráfica de la curva de aprendizaje usando el error en lugar 
    del score (como acustumbra scikit-learn)

    Parameters
    ----------
    estimator : Un estimador con mñetodos `fit`y `predict`

    X : ndarray con un shape de (n_samples, n_features)

    y : ndarray con un shape shape (n_samples) o (n_samples, n_features)

    train_sizes : ndarray de shape (n_ticks,), porcentaje de ejemplos en el
        aprendizaje en la curva de aprendizaje. Por default 5 valores entre
        el 10% y el 100% (incluidos).

    score: string con una medida de score de Scikit-learn. Consultar el manual
        para conocer los diferentes scores

    cv : int, el K-fold cross-validation. Si None, entonces K = 5

    n_jobs : int or None, numero de trabajos en paralelo. Si None entonces 1.

    """

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator, X, y, 
        scoring=score, cv=cv, n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True
    )

    if score.startswith('neg'):
        train_error_mean = -np.mean(train_scores, axis=1)
        test_error_mean = -np.mean(test_scores, axis=1)
    else:
        train_error_mean = 1 - np.mean(train_scores, axis=1)
        test_error_mean = 1 - np.mean(test_scores, axis=1)   
    train_error_std = np.std(train_scores, axis=1)
    test_error_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Ahora la graficada
    pl.figure(figsize=(18, 7))

    # Empezamos con el gráfico de la curva de aprendizaje
    pl.subplot(1, 2, 1)
    pl.plot(
      train_sizes, train_error_mean, 'o-', color="r", label="Error entrenamiento"
    )
    pl.plot(
      train_sizes, test_error_mean, 'o-', color="g", label="Error validación"
    )
    pl.fill_between(
      train_sizes, 
      train_error_mean - train_error_std,
      train_error_mean + train_error_std, 
      alpha=0.1, color="r"
    )
    pl.fill_between(
      train_sizes, 
      test_error_mean - test_error_std,
      test_error_mean + test_error_std, 
      alpha=0.1, color="g"
    )
    pl.title("Curva de aprendizaje")
    pl.xlabel("Ejemplos de entrenamiento")
    pl.ylabel("Error estimado")
    pl.grid()
    pl.legend(loc="best")

    # Ahora el número de muestras y el tiempo que le tomó procesarlas
    pl.subplot(1, 2, 2)
    pl.plot(train_sizes, fit_times_mean, 'o-')
    pl.fill_between(
      train_sizes, 
      fit_times_mean - fit_times_std,
      fit_times_mean + fit_times_std, 
      alpha=0.1
    )
    pl.xlabel("Ejemplos de entrenamiento")
    pl.ylabel("Ajuste en tiempo de computo")
    pl.title("Escalabilidad del modelo respecto a tiempos de ejecución")
    pl.grid()

    return train_sizes, train_scores, test_scores, fit_times



## Visualizando la curva de aprendizaje

**Para la Regresión rígida**

In [ ]:
plot_learning_curve(enr, X, y)
pl.show()

**Para la regresión por máquinas de vectores de soporte**

In [ ]:
plot_learning_curve(svr, X, y, cv=3, n_jobs=6)
pl.show()

**Para la regresión rígida con Kernel**

In [ ]:
plot_learning_curve(kr, X, y)
pl.show()